<a href="https://colab.research.google.com/github/daniel-hain/bibliometrics_EIST_2021/blob/master/python/BERTopic_EIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERTopic EIST 

* First tryout on EIST topic modelling using BERT an reprentation learning.


## Setup

In [ ]:
# Installing BERTopic
!pip install bertopic

1.  **Enabling the GPU.** Navigate to Edit→Notebook Settings -> select GPU from the Hardware Accelerator drop-down
2. **Install BERTTopic:** Will need to be done
3. **Restart the Notebook:**Runtime → Restart Runtime

## Data

* Abstracts of all EIST articles up to XX.2023

In [ ]:
import pandas as pd

In [ ]:
docs = pd.read_csv("https://raw.githubusercontent.com/daniel-hain/bibliometrics_EIST_2021/master/data/data_text.csv")

In [ ]:
docs.head()

,UT,PY,text
0,2-S2.0-85117715938,2021,common-pool resources and governance in sustai...
1,2-S2.0-85137288685,2022,introduction to ‘markets in sustainability tra...
2,2-S2.0-85128534377,2022,policy instruments for green-growth of cluster...
3,2-S2.0-85130369331,2022,the transition of agriculture to low carbon pa...
4,2-S2.0-85126615038,2022,hard coal phase-out and the labour market tran...


# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model. 




## Training

* We start by instantiating BERTopic. 
*We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead. 
* We will also calculate the topic probabilities. 


In [ ]:
# If you use cohere
#!pip install cohere
import cohere
cohere_api_key = "wD0eBqFNeIg3IcauiEERlTpZfuwpgDiCDrOrfsXR"

In [ ]:
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import MaximalMarginalRelevance
from bertopic.representation import KeyBERTInspired
from bertopic.representation import TextGeneration

from bertopic.representation import Cohere

from sentence_transformers import SentenceTransformer
from transformers.pipelines import pipeline

from hdbscan import HDBSCAN
from umap import UMAP

from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# EMBEDDING MODEL:
embedding_model = "allenai-specter"

# DIMENSIONALITY REDUCTION
umap_model = UMAP(n_neighbors=5, 
                  n_components=5, 
                  min_dist=0.0, 
                  metric='cosine', 
                  random_state=1337)

# CLUSTERING: resuce min_cluster_size and min_samples
hdbscan_model = HDBSCAN(min_cluster_size=5, 
                        metric='euclidean', 
                        cluster_selection_method='eom', 
                        prediction_data=True, 
                        min_samples=2)

# VECTORIZER: custom vectorizer to get rid of stopwords
vectorizer_model = CountVectorizer(stop_words="english",
                                   min_df = 10)

# TFIDF Weighting: Weighting model with redued stoppwords
ctfidf_model = ClassTfidfTransformer(bm25_weighting=True,
                                     reduce_frequent_words=True)

In [ ]:
## REPRESENTATION MODEL: KeyBErt
representation_model = KeyBERTInspired()

In [ ]:
## REPRESENTATION MODEL: Cohere
#prompt = """
#I have a social science topic that contains the following abstracts of academic journal publications: [DOCUMENTS]. 
#The topic is described by the following keywords: [KEYWORDS].
#Based on the above information, can you give a short label of the topic?
#Use between two and five words.
#"""

#co = cohere.Client(cohere_api_key)
#representation_model = Cohere(co, prompt=prompt)

In [ ]:
# BEWRTopic: specify all custom models and n_grams
topic_model = BERTopic(language="english", 
                       calculate_probabilities=True,
                       verbose=True, 
                       n_gram_range=(1, 3),
                       #
                       embedding_model=embedding_model, 
                       umap_model=umap_model,
                       hdbscan_model=hdbscan_model, 
                       vectorizer_model=vectorizer_model,
                       ctfidf_model=ctfidf_model,
                       representation_model=representation_model)

In [ ]:
topics, probs = topic_model.fit_transform(docs.loc[:,'text'])

Batches:   0%|          | 0/19 [00:00<?, ?it/s]

2023-02-16 12:13:59,125 - BERTopic - Transformed documents to Embeddings
2023-02-16 12:14:00,976 - BERTopic - Reduced dimensionality
2023-02-16 12:14:01,075 - BERTopic - Clustered reduced embeddings


In [33]:
embeddings = topic_model.embedding_model.embed_documents(docs.loc[:,'text'])

In [35]:
pd.DataFrame(embeddings).to_csv('embeddings_specter.csv')

In [ ]:
# Calculate Hirarchy
hierarchical_topics = topic_model.hierarchical_topics(docs.loc[:,'text'])

100%|██████████| 50/50 [00:06<00:00,  7.20it/s]


In [ ]:
# Calculate the topic distributions on a token-level
topic_distr, topic_token_distr = topic_model.approximate_distribution(docs.loc[:,'text'], calculate_tokens=True)

100%|██████████| 1/1 [00:01<00:00,  1.18s/it]


In [ ]:
# Calculate topics over time
topics_over_time = topic_model.topics_over_time(docs.loc[:,'text'],
                                                docs.loc[:,'PY'])

13it [00:08,  1.57it/s]


## Extracting Topics
* After fitting our model, we can start by looking at the results. 
* Typically, we look at the most frequent topics first as they best represent the collection of documents. 
* -1 refers to all outliers and should typically be ignored. 

In [ ]:
freq = topic_model.get_topic_info(); freq.head(50)

,Topic,Count,Name
0,-1,92,-1_legitimacy_governance_frameworks_geography
1,0,24,0_intermediaries_sociotechnical_legitimacy_sec...
2,1,23,1_renewables_electricity_government_netherlands
3,2,20,2_emergence_policymakers_geography_incumbent
4,3,19,3_transformative_initiatives_landscape_interve...
5,4,19,4_decarbonisation_sectoral_incumbents_firms
6,5,18,5_sector_ecological_legitimacy_practices
7,6,17,6_markets_market_frameworks_innovations
8,7,17,7_sociotechnical_landscape_institutional_regional
9,8,14,8_innovations_firms_companies_industries


## Attributes

* There are a number of attributes that you can access after having trained your BERTopic model:


| Attribute | Description |
|------------------------|---------------------------------------------------------------------------------------------|
| topics_               | The topics that are generated for each document after training or updating the topic model. |
| probabilities_ | The probabilities that are generated for each document if HDBSCAN is used. |
| topic_sizes_           | The size of each topic                                                                      |
| topic_mapper_          | A class for tracking topics and their mappings anytime they are merged/reduced.             |
| topic_representations_ | The top *n* terms per topic and their respective c-TF-IDF values.                             |
| c_tf_idf_              | The topic-term matrix as calculated through c-TF-IDF.                                       |
| topic_labels_          | The default labels for each topic.                                                          |
| custom_labels_         | Custom labels for each topic as generated through `.set_topic_labels`.                                                               |
| topic_embeddings_      | The embeddings for each topic if `embedding_model` was used.                                                              |
| representative_docs_   | The representative documents for each topic if HDBSCAN is used.                                                |

For example, to access the predicted topics for the first 10 documents, we simply run the following:

In [ ]:
topic_model.topics_[:10]

[-1, 6, 12, 28, -1, 29, 23, 19, 8, 45]

# **Visualization**
* There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. 
* Topic modeling is, to a certain extent, quite subjective. * Visualizations help understand the topics that were created. 

## General topic relation

In [ ]:
topic_model.visualize_topics()

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [ ]:
topic_model.visualize_barchart(top_n_topics=50, n_words = 15)

In [38]:
topic_model.visualize_documents(docs.loc[:,'text'], embeddings=embeddings)

## Visualize Topic Hierarchy

* The topics that were created can be hierarchically reduced. 
* In order to understand the potential hierarchical structure of the topics, we can create clusters and visualize how they relate to one another. 
* This might also help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [ ]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [37]:
# Run the visualization with the original embeddings
topic_model.visualize_hierarchical_documents(docs.loc[:,'text'], hierarchical_topics, embeddings=embeddings)

## Topics over time

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [ ]:
topic_model.visualize_heatmap(n_clusters=5, width=1000, height=1000)

## Visualize Topic Probabilities

- The variable `probabilities` that is returned from `transform()` or `fit_transform()` can  be used to understand how confident BERTopic is that certain topics can be found in a document. 



In [ ]:
# Select a document
doc_nr = 102
docs.loc[doc_nr,'text']

'influences of technological and sectoral contexts on technological innovation systems. this paper analyses how technological and sectoral context structures influence the functional pattern of a focal technological innovation system (tis), focusing on value chain interdependencies. through a case study of the ‘embryonic’ bio-succinate from mixed food waste tis in sweden, we show that all identified context structures exert both negative and positive influences on tis functions by influencing resource availability and accessibility, market conditions and the wider selection environment. contextual influences result from interdependencies throughout the value chain, but in contrast to previous studies, direct structural overlaps are not as relevant as competition for resources, markets and policy support. competition does not only come from the regime but also from other related tiss and sectors. the study also confirms the importance of contextual influences from the international tis.

In [ ]:
# Get the probability distribution
topic_model.visualize_distribution(probs[doc_nr], min_probability=0.01)

In [ ]:
# Visualize the token-level distributions
doc_topic_dist = topic_model.visualize_approximate_distribution(docs.loc[doc_nr,'text'], topic_token_distr[doc_nr])
doc_topic_dist